# House Prices Prediction using TensorFlow Decision Forests

决策森林是一类基于树的模型，包括随机森林和梯度提升树。
在处理表格数据时，它们是最好的起点，并且在开始使用深度神经网络之前，它们通常会优于其他模型(或提供一个强大的基准)。


In [ ]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Comment this if the data visualisations doesn't work on your side
%matplotlib inline

In [ ]:
print("TensorFlow v" + tf.__version__)
print("TensorFlow Decision Forests v" + tfdf.__version__)

## 观察数据集


In [ ]:
train_file_path = "../input/house-prices-advanced-regression-techniques/train.csv"
dataset_df = pd.read_csv(train_file_path)
print("Full train dataset shape is {}".format(dataset_df.shape))

数据由81列和1460项组成。用下面的代码打印出数据集的前3个元素，可以看到数据集的所有81维:

In [ ]:
dataset_df.head(3)

共有79个特征列。
使用这些特征训练模型，模型的目标是预测` salesprice `列所表示的房屋销售价格。


这里删除`Id`列，因为它对模型训练不是必需的。

In [ ]:
dataset_df = dataset_df.drop('Id', axis=1)
print(dataset_df.shape)
dataset_df.head(3)

可以使用以下代码来检查特征列的类型:

In [ ]:
dataset_df.info()

### 房价分布
 
这里查看房价是如何分布的。

In [ ]:
print(dataset_df['SalePrice'].describe())
plt.figure(figsize=(9, 8))
sns.distplot(dataset_df['SalePrice'], color='g', bins=100, hist_kws={'alpha': 0.4});

### 数值数据分布
查看数值特征是如何分布的。
首先列出数据集中所有类型的数据，并只选择数值类型。

In [ ]:
list(set(dataset_df.dtypes.tolist()))

In [ ]:
df_num = dataset_df.select_dtypes(include = ['float64', 'int64'])
df_num.head()

绘制所有数值特征的分布

In [ ]:
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

## 准备数据集
该数据集混合了数值型、类别型和缺失型特征。
TF-DF原生支持所有这些特征类型，不需要预处理。
这是基于树的模型的一个优点，使其成为Tensorflow和ML的一个很好的入口。

将数据集分成训练集和验证集

In [ ]:
import numpy as np

def split_dataset(dataset, test_ratio=0.30):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, valid_ds_pd = split_dataset(dataset_df)

print(f"{len(dataset_df)} examples in training datasets.")
print(f"{len(train_ds_pd)} examples in training, {len(valid_ds_pd)} examples in testing.")

在训练模型之前，还需要一步。
需要将数据集从Pandas格式(`pd.DataFrame`)转换为TensorFlow数据集格式(`tf.data.Dataset`)。

默认情况下，随机森林模型被配置为训练分类任务。由于这是一个回归问题，我们将在这里指定任务的类型(`tfdf.keras.Task.REGRESSION`)作为参数。

In [ ]:
label = 'SalePrice'
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task = tfdf.keras.Task.REGRESSION)
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label=label, task = tfdf.keras.Task.REGRESSION)

## 模型选择

有几种基于树的模型可供选择:
- RandomForestModel 
- GradientBoostedTreesModel 
- CartModel 
- DistributedGradientBoostedTreesModel 
 
首先，这里使用随机森林。
这是最著名的决策森林训练算法。 
 
随机森林是一组决策树的集合，每棵决策树都在训练数据集的一个随机子集上独立训练(用替换进行采样)。
该算法具有对过拟合鲁棒性强、简单易用等特点。

We can list the all the available models in TensorFlow Decision Forests using the following code:

In [ ]:
tfdf.keras.get_all_models()

## 模型超参数如何配置？


TensorFlow决策森林为您提供了良好的默认值(例如，在我们的基准上排名最高的超参数，略微修改以在合理的时间内运行)。如果你想配置学习算法，你会发现有很多选项可以探索以获得尽可能高的精度。

可以选择模板，也可以设置如下参数:
```rf = tfdf.keras.RandomForestModel(hyperparameter_template="benchmark_rank1", task=tfdf.keras.Task.REGRESSION)```

Read more [here](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/RandomForestModel).

## 创建一个随机森林

Today, we will use the defaults to create the Random Forest Model while specifiyng the task type as `tfdf.keras.Task.REGRESSION`.

In [ ]:
rf = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
rf.compile(metrics=["mse"]) # Optional, you can use this to include a list of eval metrics

## 训练模型

In [ ]:
rf.fit(x=train_ds)

## 可视化模型
基于树的模型的一个好处是可以很容易地将它们可视化。随机森林中使用的树的默认数量是300。

In [ ]:
tfdf.model_plotter.plot_model_in_colab(rf, tree_idx=0, max_depth=3)

## 在Out of bag (OOB)数据和验证数据集上评估模型


在训练数据集之前，我们手动分离了30%的数据集进行验证，命名为`valid_ds`。 
 
我们还可以使用Out of bag (OOB)分数来验证我们的RandomForestModel。 
在训练随机森林模型时，该算法从训练集中选取一组随机样本，用其余样本对模型进行微调。未选择的数据子集称为袋外数据(Out of bag data, OOB)。 
OOB得分是根据OOB数据计算的。

更多关于 OOB data [参考资料](https://developers.google.com/machine-learning/decision-forests/out-of-bag).

训练日志显示了模型中树的数量在out-of-bag数据集上评估的均方根误差(RMSE)。
我们来画一下。 
 
注意:这个超参数的值越小越好。

In [ ]:
import matplotlib.pyplot as plt
logs = rf.make_inspector().training_logs()
plt.plot([log.num_trees for log in logs], [log.evaluation.rmse for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("RMSE (out-of-bag)")
plt.show()

还可以看到OOB数据集的一些一般统计数据


In [ ]:
inspector = rf.make_inspector()
inspector.evaluation()

现在，这里使用验证数据集进行评估。

In [ ]:
evaluation = rf.evaluate(x=valid_ds,return_dict=True)

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

## 变量重要性

变量重要性通常表示特征对模型预测或质量的贡献程度。
使用TensorFlow决策森林有几种方法来识别重要的特征。 
让我们列出决策树中可用的`Variable Importances`:

In [ ]:
print(f"Available variable importances:")
for importance in inspector.variable_importances().keys():
  print("\t", importance)

例如，让我们展示变量重要性`NUM_AS_ROOT`的重要特征。 
 
`NUM_AS_ROOT`的重要性得分越大，它对模型结果的影响越大。 
 
默认情况下，列表按照从小到大的顺序排列。
从输出可以推断，在随机森林中，与其他特征相比，列表顶部的特征在大多数树中被用作根节点。

In [ ]:
inspector.variable_importances()["NUM_AS_ROOT"]

使用Matplotlib绘制变量的重要性

In [ ]:
plt.figure(figsize=(12, 4))

# Mean decrease in AUC of the class 1 vs the others.
variable_importance_metric = "NUM_AS_ROOT" # SUM_SCORE, NUM_AS_ROOT
variable_importances = inspector.variable_importances()[variable_importance_metric]

# Extract the feature name and importance values.
#
# `variable_importances` is a list of <feature, importance> tuples.
feature_names = [vi[0].name for vi in variable_importances]
feature_importances = [vi[1] for vi in variable_importances]
# The feature are ordered in decreasing importance value.
feature_ranks = range(len(feature_names))

bar = plt.barh(feature_ranks, feature_importances, label=[str(x) for x in feature_ranks])
plt.yticks(feature_ranks, feature_names)
plt.gca().invert_yaxis()

# TODO: Replace with "plt.bar_label()" when available.
# Label each bar with values
for importance, patch in zip(feature_importances, bar.patches):
  plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{importance:.4f}", va="top")

plt.xlabel(variable_importance_metric)
plt.title("NUM AS ROOT of the class 1 vs the others")
plt.tight_layout()
plt.show()

## 结果
最后利用该模型对竞赛测试数据进行预测。

In [ ]:
test_file_path = "../input/house-prices-advanced-regression-techniques/test.csv"
test_data = pd.read_csv(test_file_path)
ids = test_data.pop('Id')

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    test_data,
    task = tfdf.keras.Task.REGRESSION
)

preds = rf.predict(test_ds)
output = pd.DataFrame({
    'Id': ids,
    'SalePrice': preds.squeeze()
})

output.head()


In [ ]:
sample_submission_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample_submission_df['SalePrice'] = rf.predict(test_ds)
sample_submission_df.to_csv('submission.csv', index=False)
sample_submission_df.head()